<a href="https://colab.research.google.com/github/gabrielfernandorey/ITBA-NLP/blob/main/ITBA_nlp01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico NLP - Detección de Tópicos y clasificación
- ITBA 2024
- Alumno: Gabriel Rey
---

### Resumen del problema

- Calcular los tópicos de portales de noticias que se reciben 
- Frecuencia del cálculo de tópicos: diaria
- Colección de noticias: diariamente, en lotes o de a un texto.
- Identificar tópicos, entidades, keywords y análisis de sentimiento.

### Datos
- Se reciben las noticias con formato: Titulo, Texto, Fecha, Entidades, Keywords

### Tareas
- Modelo de detección de tópicos diario utilizando embeddings
- Definir un criterio de agrupación de tópicos aplicado al mismo día y entre distintos días (merging)
- Almacenar los embeddings de tópicos en una base de datos vectorial
- Modelo de datos dado: 
    - Id del tópico
    - Nombre del tópico
    - Keywords
    - Embbeding
    - Fecha de creación
    - Fecha de entrenamiento inicial
    - Fecha de entrenamiento actualizada
    - Umbral de detección
    - Documento mas cercano
---
Tareas en esta notebook:
- Inicializar la base de datos vectorial
- Ingestar data
- NER: Encontrar las entidades de cada documento
- Limpiar data
- Modelo: Armado del modelo BERTopic
- Entrenamiento
- Almacenamiento en base de datos vectorial


### NLP_01_data
Esta notebook se utiliza para:
- obtener un lote predefinido de las noticias de una fecha dada
- se realiza el ejercicio de obtención de entities y keywords, mas allá de tenerlas provistas en el set de datos de origen.
- se obtiene el vocabulario para utilizar en BERTopic

Esta y las consecuentes notebooks son el desarrollo de base de procesos y funciones para la web app provista.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import re
import json
from datetime import datetime
from dotenv import load_dotenv
from tqdm import tqdm
from collections import Counter

import spacy

from NLP_tools import clean_all
from core.functions import *

# -->> levantar la base antes de ejecutar
from opensearch_data_model import os_client
from opensearch_io import init_opensearch

from datasets import load_dataset

### Inicializamos la base vectorial
- Se modifica el indice de la base "Topic" agregando referencias del documento mas cercano como el ID y el titulo
- Se crea un nuevo indice para las noticias

In [2]:
# Inicialización de indices
init_opensearch()

Indice Topic creado
Indice News creado


### Path

In [3]:
load_dotenv()
PATH_REMOTO='/content/ITBA-NLP/data/'
PATH=os.environ.get('PATH_LOCAL', PATH_REMOTO)
PATH

'C:/Users/gabri/OneDrive/Machine Learning/Github/ITBA-NLP/data/'

### Obtenemos los datos 

In [30]:
# Read the parquet file 

# Desde Hugginface
date_choice = '2024-07-17'  # formato aaaa-mm-dd
path_file = f"jganzabalseenka/news_{date_choice}_24hs"
dataset = load_dataset(path_file)
df_parquet = pd.DataFrame(dataset['train'])

# Local """ el formato del archivo parque debe ser fecha aaaammdd "
# date_choice = '2024-07-15' 
# path_file = f"{date_choice}.parquet"
# df_parquet = pd.read_parquet(PATH+path_file)

df_parquet.head(1)

,asset_id,title_ch,Asset Destination,media,impact,start_time_utc,start_time_local,entities_curated,entities,predicted_at_entities,entities_raw_transformers,entities_transformers,title,text,keywords,predicted_at_keywords,truncated_text,title_and_text,prediction_delay_predictions,prediction_delay
0,114894323,Supremo recibe expediente de la Fiscalía colom...,http://infobae.com/america/agencias/2024/07/16...,Infobae,9642,2024-07-17 00:20:49,2024-07-16 21:20:49,"[Fiscalía General de la Nación Colombia, Alian...","[Iván Name, Julián Peinado Ramírez, Andrés Cal...",2024-07-17 00:59:39.937401,"[{'entities': [{'end': 7, 'entity_group': 'ORG...","[Supremo, Fiscalía colo, Newsroom Infobae Nuev...",Supremo recibe expediente de la Fiscalía colom...,"16 Jul, 2024 Por Newsroom Infobae Nuevo Bogot...","[los congresistas, fiscalía colombiana, falsed...",2024-07-17 01:05:59.174600,"16 Jul, 2024 Por Newsroom Infobae Nuevo Bogot...",Supremo recibe expediente de la Fiscalía colom...,0.105344,0.752826


In [31]:
# Cantidad total de documentos
len(df_parquet)

16037

### Validar datos para la fecha

In [32]:
choice = "".join(date_choice.split('-'))
df_parquet.sort_values("start_time_local", ascending=True, inplace=True)
df_date_filtered = df_parquet[df_parquet['start_time_local'].dt.date == pd.to_datetime(choice).date()]
print(f"Registros para la fecha {choice} -> {len(df_date_filtered)} de un total de {len(df_parquet)}")

Registros para la fecha 20240717 -> 11663 de un total de 16037


### Obtenemos un lote de N noticias por dia (para agilizar el procesamiento)

In [33]:
batch_news=os.environ.get('BATCH_NEWS', 1000)

df_batch = df_date_filtered.sample(n=int(batch_news))
data = list(df_batch['text'])
df_batch.head(1)

,asset_id,title_ch,Asset Destination,media,impact,start_time_utc,start_time_local,entities_curated,entities,predicted_at_entities,entities_raw_transformers,entities_transformers,title,text,keywords,predicted_at_keywords,truncated_text,title_and_text,prediction_delay_predictions,prediction_delay
12619,114944575,Estudiantes de La Plata y Central Córdoba van ...,http://dobleamarilla.com.ar/liga-/estudiantes-...,Doble Amarilla,1022,2024-07-17 12:42:29,2024-07-17 09:42:29,"[Club Estudiantes de La Plata, Eduardo Domíngu...","[Eros Mancuso, Luciano Lollo, Belgrano, Gastón...",2024-07-17 12:49:00.219674,"[{'entities': [{'end': 23, 'entity_group': 'OR...","[Estudiantes de La Plata, Central Córdoba, Est...",Estudiantes de La Plata y Central Córdoba van ...,Estudiantes de La Plata y Central Córdoba de S...,"[octavos, eros, plata, estadio, monte maíz, pu...",2024-07-17 12:49:36.524530,Estudiantes de La Plata y Central Córdoba de S...,Estudiantes de La Plata y Central Córdoba van ...,0.010085,0.118757


In [34]:
# Grabar a disco el dataframe filtrado 
df_batch.to_parquet(PATH+choice+".parquet")

### StopWords
Se genera una lista especial de stopwords

In [9]:
# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)
SPANISH_STOPWORDS_SPECIAL = list(pd.read_csv(PATH+'spanish_stop_words_spec.csv' )['stopwords'].values)

In [10]:
""" import csv
# Guardar la lista de stopwords especial en un archivo CSV
with open(PATH+"spanish_stop_words_spec.csv", mode='w', newline='', encoding='utf-8') as archivo:
    escritor = csv.writer(archivo)
    escritor.writerow(['stopwords'])
    for stopword in SPANISH_STOPWORDS_SPECIAL:
        escritor.writerow([stopword]) """

' import csv\n# Guardar la lista de stopwords especial en un archivo CSV\nwith open(PATH+"spanish_stop_words_spec.csv", mode=\'w\', newline=\'\', encoding=\'utf-8\') as archivo:\n    escritor = csv.writer(archivo)\n    escritor.writerow([\'stopwords\'])\n    for stopword in SPANISH_STOPWORDS_SPECIAL:\n        escritor.writerow([stopword]) '

### NER - Named Entity Recognition
Obtener entidades de las noticias 

-   Nota: Aunque el dataset original provee keywords y entities, se realiza el proceso de generacion propia de keywords y entites y se utilizan para modelar. 

In [11]:
# Cargar el modelo de spaCy para español
spa = spacy.load("es_core_news_lg")

In [ ]:
# Cargar o saltar carga y procesar celda inferior
#with open(PATH+f'modelos/entities_{str(choice)}.json', 'r') as json_file:
#    entities = json.load(json_file)

#with open(PATH+f'modelos/keywords_{str(choice)}.json', 'r') as json_file:
#    keywords = json.load(json_file)

#with open(PATH+f'modelos/vocabulary_{str(choice)}.json', 'r') as json_file:
#    vocab = json.load(json_file) 

In [35]:
# Detectar entidades para todos los documentos usando spaCy
# se procesa utilizando un criterio de seleccion reflejado en el codigo

entities = []
for data_in in tqdm(data):

    # Contabilizar palabras en doc original
    normalized_text = re.sub(r'\W+', ' ', data_in.lower())
    words_txt_without_stopwords = [word for word in normalized_text.split() if word not in SPANISH_STOPWORDS+SPANISH_STOPWORDS_SPECIAL]
    words_txt_counter = Counter(words_txt_without_stopwords)
    words_counter = {elemento: cuenta for elemento, cuenta in sorted(words_txt_counter.items(), key=lambda item:item[1], reverse=True) if cuenta > 1}

    # Extraer entidades del doc segun atributos
    extract = spa(data_in)
    entidades_spacy = [(ent.text, ent.label_) for ent in extract.ents]
    ent_select = [ent for ent in entidades_spacy if ent[1] == 'PER' or ent[1] == 'ORG' or ent[1] == 'LOC' ]

    # Extraer entidades de "maximo 3 palabras"
    ent_max_3 = [ent[0] for ent in ent_select if len(ent[0].split()) <= 3]
    ent_clean = clean_all(ent_max_3, accents=False)
    ent_unique = list(set([ word for word in ent_clean if word not in SPANISH_STOPWORDS+SPANISH_STOPWORDS_SPECIAL] ))

    ents_proc = {}
    for ent in ent_unique:
        
        # Criterio de selección 
        weight = 0
        for word in ent.split():
            if word in words_counter:
                weight += 1 /len(ent.split()) * words_counter[word]
        
        ents_proc[ent] = round(weight,4)

    ents_proc_sorted = {k: v for k, v in sorted(ents_proc.items(), key=lambda item: item[1], reverse=True) if v > 0}

    # Crear la lista preliminar de entidades procesadas por noticia 
    pre_entities = [key for key, _ in ents_proc_sorted.items()] 

    # Obtener las últimas palabras de cada entidad que tenga mas de una palabra por entidad
    last_words = list(set([ent.split()[-1] for ent in pre_entities if len(ent.split()) > 1 ]))

    # Eliminar palabra única si la encuentra al final de una compuesta
    pre_entities_without_last_word_equal = []
    for idx, ent in enumerate(pre_entities):
        if not (len(ent.split()) == 1 and ent in last_words):
            pre_entities_without_last_word_equal.append(ent)

    # Obtener las palabras únicas
    unique_words = [ ent.split()[0] for ent in pre_entities_without_last_word_equal if len(ent.split()) > 1 ]

    # Eliminar palabra única si la encuentra al comienzo de una compuesta
    pre_entities_without_first_word_equal = []
    for idx, ent in enumerate(pre_entities_without_last_word_equal):
        if not (len(ent.split()) == 1 and ent in unique_words):
            pre_entities_without_first_word_equal.append(ent)

    # obtener entidades filtradas
    if len(pre_entities_without_first_word_equal) > 10:
        umbral = 10 + (len(pre_entities_without_first_word_equal)-10) // 2
        filter_entities = pre_entities_without_first_word_equal[:umbral] 
    else:
        filter_entities = pre_entities_without_first_word_equal[:10]

    pre_original_entities = []
    # capturar las entidades en formato original
    for ent in filter_entities:
        pre_original_entities.append([elemento for elemento in ent_max_3 if elemento.lower() == ent.lower()])

    sort_original_entities = sorted(pre_original_entities, key=len, reverse=True)
    
    try:
        entities.append( [ent[0] for ent in sort_original_entities if ent] ) 
    except Exception as e:
        entities.append([])


  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [00:34<00:00, 14.36it/s]


In [36]:
len(entities)

500

In [37]:
# Grabar
with open(PATH+f'preproc_notebook/entities_{str(choice)}.json', 'w') as file:
    json.dump(entities, file)

## Keywords
Obtener palabras clave de las noticias

In [38]:
# Detectar keywords para todos los documentos usando spaCy

keywords_spa = []
for doc in tqdm(data):
    extract = spa(doc)
    keywords_spa.append([(ext.text, ext.pos_) for ext in extract])  

  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [00:30<00:00, 16.65it/s]


### Keyboards with neighboards
- Se seleccionan keywords unigrama y bigrama mediante la funcion keywords_with_neighboards(), que a su vez llama a las funciones get_bigrams() y get_neighbor_words()

In [16]:
# Funcion para obtener keywords con combinaciones de bigramas
def get_bigrams(word_list, number_consecutive_words=2):
    
    ngrams = []
    adj_length_of_word_list = len(word_list) - (number_consecutive_words - 1)
    
    for word_index in range(adj_length_of_word_list):
        
        # Indexar la lista 
        ngram = word_list[word_index : word_index + number_consecutive_words]
        
        # Agregar a la lista de "ngrams"
        ngrams.append(ngram)
        
    return ngrams

In [17]:
# devolver las palabras más frecuentes que aparecen junto a una palabra clave en particular
def get_neighbor_words(keyword, bigrams, pos_label = None):
    
    neighbor_words = []
    keyword = keyword.lower()
    
    for bigram in bigrams:
        
        # Extrae solo las palabras en minúsculas (no las etiquetas) para cada bigrama
        words = [word.lower() for word, label in bigram]        
        
        # Comprueba si la palabra clave está en el bigram
        if keyword in words:
            idx = words.index(keyword)
            for word, label in bigram:
                
                #Ahora nos centramos en la palabra vecina, no en la palabra clave
                if word.lower() != keyword:
                    #Si la palabra vecina coincide con la pos_label correcta, agregarla a la lista maestra
                    if label == pos_label or pos_label == None:
                        if idx == 0:
                            neighbor_words.append(" ".join([keyword, word.lower()]))
                        else:
                            neighbor_words.append(" ".join([word.lower(), keyword]))
                    
    return Counter(neighbor_words).most_common()

In [18]:
def keywords_with_neighboards(keywords_spa, POS_1='NOUN', POS_2='ADJ'):
    """
    Funcion que devuelve dos listas:
    - lista de keywords with neighboards (segun argumentos POS_1 y POS_2)
    - lista de keywords mas frecuentes (segun argumentos POS_1 y POS_2)
    """

    doc_kwn = []
    commons = []
    for keywords in keywords_spa:
    
        # Obtenemos las keywords del tipo (Universal Dependences) mas frecuentes de cada doc (spaCy format)
        words = []
        for k_spa in keywords:
            if k_spa[1] == POS_1:
                words.append(k_spa[0])

        cont_words = Counter(words)

        common = cont_words.most_common()
        commons.append( [com for com in common if com[1] > 1] )

        # Calcular un umbral de corte (en repeticiones) para los keywords obtenidos
            ## suma de todos los valores
        valores = [valor for _, valor in common]

            ## Calcular los pesos como proporcionales a los valores mismos
        pesos = np.array(valores) / np.sum(valores)

            ## Calcular el umbral ponderado, valor 2 o superior ( debe repetirse la keyword al menos una vez )
        threshold = max(2, round(np.sum(np.array(valores) * pesos),4))


        # Obtenemos los bigramas del doc        
        tokens_and_labels = [(token[0], token[1]) for token in keywords if token[0].isalpha()]

        bigrams = get_bigrams(tokens_and_labels)

        keywords_neighbor = []
        for item_common in common:
            if item_common[1] >= threshold or len(keywords_neighbor) < 6: # corte por umbral o menor a 6
                
                kwn = get_neighbor_words(item_common[0], bigrams, pos_label=POS_2)
                if kwn != []:
                    keywords_neighbor.append( kwn )

        sorted_keywords_neighbor = sorted([item for sublist in keywords_neighbor for item in sublist ], key=lambda x: x[1], reverse=True)
        
        doc_kwn.append(sorted_keywords_neighbor)

    return doc_kwn, commons

In [39]:
# obtenemos keywords with neighboards y keywords mas frecuentes
k_w_n, keyword_single = keywords_with_neighboards(keywords_spa)

In [40]:
# muestra de keywords with neighboards
k_w_n[4]

[('redes sociales', 2),
 ('contrato legal', 1),
 ('contrato ligado', 1),
 ('nuevo contrato', 1),
 ('momento ideal', 1),
 ('manera rápida', 1),
 ('importante marca', 1),
 ('contenido erótico', 1),
 ('alto contenido', 1),
 ('segunda vez', 1),
 ('reglas incumplidas', 1)]

In [41]:
# filtramos las que al menos se repiten una vez
filtered_k_w_n = [ [tupla[0] for tupla in sublista if tupla[1] > 1] for sublista in k_w_n ]

In [42]:
# muestra
filtered_k_w_n[1]

['empresariado sanmartinense',
 'sector laboral',
 'verano pasado',
 'empresarios hoteleros',
 'reclamo justo',
 'trabajadores hoteleros']

In [43]:
# Analizamos los keywords unigrama
keyword_single[1]

[('familias', 11),
 ('verano', 10),
 ('empresarios', 9),
 ('reclamo', 7),
 ('trabajadores', 7),
 ('julio', 6),
 ('gastronómicos', 6),
 ('oportunidad', 6),
 ('empresariado', 6),
 ('par', 6),
 ('negativa', 6),
 ('beneficio', 6),
 ('sector', 6),
 ('pago', 6),
 ('necesidad', 6),
 ('cámara', 5),
 ('postura', 5),
 ('acuerdo', 5),
 ('intereses', 5),
 ('adelanto', 5),
 ('aumento', 5),
 ('temporada', 5),
 ('inflación', 5),
 ('lleno', 5),
 ('salarios', 5),
 ('meses', 5),
 ('invierno', 5),
 ('agosto', 5),
 ('enero', 5),
 ('febrero', 5),
 ('escenario', 5),
 ('referentes', 5),
 ('respuesta', 5),
 ('estrategia', 5),
 ('tiempo', 5),
 ('razón', 4),
 ('vez', 4),
 ('reflexión', 4),
 ('localidad', 4),
 ('actitud', 4),
 ('fe', 4),
 ('hora', 4),
 ('relación', 4),
 ('esfuerzo', 3),
 ('dirigentes', 3),
 ('cuerpo', 3),
 ('delegados', 3),
 ('secretarios', 3),
 ('punto', 3),
 ('vista', 3),
 ('condiciones', 3),
 ('clima', 3),
 ('región', 3),
 ('caso', 3),
 ('vidas', 3),
 ('compañeros', 3),
 ('salvo', 3),
 ('repr

In [44]:
# Si un keyword unigrama coincide en los bigramas elegidos se descarta
# la cantidad de keywords se obtiene utilizando la media como umbral de corte

# Umbral
values = [value for sublist in keyword_single for _, value in sublist]
threshold = np.mean(values)

for i, sublist in enumerate(keyword_single):
    lista_k_w_n = list(set([word for sentence in filtered_k_w_n[i] for word in sentence.split()]))
    for tupla in sublist:
        if tupla[1] >= threshold and tupla[0] not in lista_k_w_n:
            filtered_k_w_n[i].append(tupla[0])

keywords = filtered_k_w_n      

In [45]:
keywords[1]

['empresariado sanmartinense',
 'sector laboral',
 'verano pasado',
 'empresarios hoteleros',
 'reclamo justo',
 'trabajadores hoteleros',
 'familias',
 'julio',
 'gastronómicos',
 'oportunidad',
 'par',
 'negativa',
 'beneficio',
 'pago',
 'necesidad',
 'cámara',
 'postura',
 'acuerdo',
 'intereses',
 'adelanto',
 'aumento',
 'temporada',
 'inflación',
 'lleno',
 'salarios',
 'meses',
 'invierno',
 'agosto',
 'enero',
 'febrero',
 'escenario',
 'referentes',
 'respuesta',
 'estrategia',
 'tiempo',
 'razón',
 'vez',
 'reflexión',
 'localidad',
 'actitud',
 'fe',
 'hora',
 'relación',
 'esfuerzo',
 'dirigentes',
 'cuerpo',
 'delegados',
 'secretarios',
 'punto',
 'vista',
 'condiciones',
 'clima',
 'región',
 'caso',
 'vidas',
 'compañeros',
 'salvo']

In [46]:
# Grabar
with open(PATH+f'preproc_notebook/keywords_{choice}.json', 'w') as file:
    json.dump(keywords, file)

#### BOW - Armado del vocabulario con las entidades y keywords

In [47]:
# Unificar Entities + Keywords + Keywords with neighboards
vocab = list(set().union(*entities, *keywords))
len(vocab)

3646

In [48]:
# Guardar vocabulario
with open(PATH+f'preproc_notebook/vocabulary_{choice}.json', 'w') as file:
    json.dump(vocab, file)

### Guardar noticias en el indice news de la base

In [49]:
# configurar  batch_size = ( ej.: 5000 ) si se supera el limite 100MB en elasticsearch por operacion
index_name = 'news'
bulk_data = []

for idx, text_news in tqdm(enumerate(data)):
    doc = {
        'index': {
            '_index': index_name,
            '_id': int(df_batch.iloc[idx]['asset_id'])
        }
    }
    reg = {
        'title': str(df_batch.iloc[idx].title),
        'news' : str(text_news), 
        'author': str(df_batch.iloc[idx]['media']),
        'vector': None,
        'keywords' : keywords[idx],
        'entities' : entities[idx],
        'created_at': parse(str(df_batch.iloc[idx]['start_time_local'])).isoformat(),
        'process': False
    }
    bulk_data.append(json.dumps(doc))
    bulk_data.append(json.dumps(reg))

# Convertir la lista en un solo string separado por saltos de línea
bulk_request_body = '\n'.join(bulk_data) + '\n'

# Enviar la solicitud bulk
response = os_client.bulk(body=bulk_request_body)

if response['errors']:
    print("Errores encontrados al insertar los documentos")
else:
    print("Documentos insertados correctamente")


0it [00:00, ?it/s]

500it [00:00, 887.29it/s]


Documentos insertados correctamente


#### Funcion para comparar resultados entre keywords y entities provistas y las generadas

In [51]:
def funcion_aux(ID):
    keywords_df = df_parquet[df_parquet.asset_id==ID]['keywords'].values[0]
    entities_df = df_parquet[df_parquet.asset_id==ID]['entities'].values[0]
    print(f"Noticia ID: {ID} {df_parquet[df_parquet.asset_id==ID]['title'].values}\n")
    print(f"Entities de dataframe: {entities_df}")
    print(f"Keywords de dataframe: {keywords_df}")
    print("-"*80)
    condicion = df_batch['asset_id'] == ID
    posicion = df_batch.index[condicion].tolist()[0]
    posicion_ordinal = df_batch.index.get_loc(posicion)
    print(f"Entities generadas: {entities[posicion_ordinal]}")
    print(f"Keywords generadas: {filtered_k_w_n[posicion_ordinal]}")


ID = df_batch.iloc[np.random.randint(len(df_batch))]['asset_id']
funcion_aux(ID)
print(f"\n {df_batch[df_batch.asset_id == ID].text.values}")

Noticia ID: 114954707 ['Murió Nerón, uno de los perros que encontró rastros de Loan Danilo Peña en el auto de Caillava y Pérez']

Entities de dataframe: ['Loan', 'Carlos Pérez', 'Nerón', 'Brigada K9', 'Federación Entrerriana de Asociaciones de Bomberos Voluntarios']
Keywords de dataframe: ['rastros', 'labrador', 'los perros', 'numerosas misiones', 'asociaciones', 'búsqueda', 'color marrón', 'media asta', 'cuartel', 'certificación', 'fiebre', 'amplio historial', 'infección', 'rescate', 'fallecimiento', 'autos', 'homenaje', 'compañero', 'personas', 'registro nacional', 'duelo', 'regreso', 'reconocimiento', 'dudas', 'foto instagram', 'triste noticia', 'diálogo', 'localidad', 'equipos', 'repentina muerte', 'participación', 'servicio']
--------------------------------------------------------------------------------
Entities generadas: ['Nerón', 'Pabellón Nacional', 'Provincial', 'provincia de Corrientes', 'Loan Danilo Peña', '9 de Julio', 'Canal 9 Litoral']
Keywords generadas: ['color marró

## Resumen

El criterio de selección de entities y keywords se basa en seleccionar solo las que han sido mayormente nombradas, sin embargo es solo un criterio, y podrían obtenerse una mayor cantidad o la totalidad ajustando el criterio de seleccion.